In [ ]:
import tensorflow 
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D,MaxPooling2D, LeakyReLU, PReLU 

In [ ]:
def read_single_tfrecord(tfrecord_file, batch_size, net):
    # generate a input queue
    # each epoch shuffle
    filename_queue = tf.string_input_producer([tfrecord_file],shuffle=True)
    # read tfrecord
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    image_features = tf.parse_single_example(
        serialized_example,
        features={
            'image/encoded': tf.FixedLenFeature([], tf.string),#one image  one record
            'image/label': tf.FixedLenFeature([], tf.int64),
            'image/roi': tf.FixedLenFeature([4], tf.float32),
            'image/landmark': tf.FixedLenFeature([10],tf.float32)
        }
    )
    if net == 'PNet':
        image_size = 12
    elif net == 'RNet':
        image_size = 24
    else:
        image_size = 48
    image = tf.decode_raw(image_features['image/encoded'], tf.uint8)
    image = tf.reshape(image, [image_size, image_size, 3])
    image = (tf.cast(image, tf.float32)-127.5) / 128
    
    # image = tf.image.per_image_standardization(image)
    label = tf.cast(image_features['image/label'], tf.float32)
    roi = tf.cast(image_features['image/roi'],tf.float32)
    landmark = tf.cast(image_features['image/landmark'],tf.float32)
    image, label,roi,landmark = tf.train.batch(
        [image, label,roi,landmark],
        batch_size=batch_size,
        num_threads=2,
        capacity=1 * batch_size
    )
    label = tf.reshape(label, [batch_size])
    roi = tf.reshape(roi,[batch_size,4])
    landmark = tf.reshape(landmark,[batch_size,10])
    return image, label, roi,landmark

In [ ]:
#Class PNet():
    #P-Net


    # Data Preprocessing
    # before passing your data into a neural network you normalize the data by Scaling it

    # scaling the data 
    #X = X/255.0

    #you can also use tf.keras.utils.normalize(X) 
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(64, activation='relu')(x)
    predictions = layers.Dense(10, activation='softmax')(x)
    # Convoluted neural network

    def p_net():
        
        model = Sequential() #there are 2 types of models but this is the most common
        model.add(Conv2D(10,(3, 3), strides=1, name='conv1',padding='SAME', input_shape =(12, 12, 3) ) )
        model.add(PReLU(name='prelu1'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

        model.add(Conv2D(16,(3, 3), strides=1,name='conv2', input_shape =(5, 5, 10)))
        model.add(PReLU(name='prelu2'))

        model.add(Conv2D(32,(3,3),strides=1,name='conv3'))
        model.add(PReLU(name='prelu3'))

        model.add(Conv2D(2, (1, 1), activation='softmax',name='classifier1'))
        model.add(PReLU(name='prelu4'))

        model.add(Dense(4,name='bbox_regression'))

        model.add(Dense(10,name='landmark'))
        
        #model = tf.keras.Model(inputs=, outputs=)
        model.compile(optimizer=tf.optimizers.Adam(0.01), loss='categorical_crossentropy', metrics=['accuracy'])
        
        #my_adam = adam(lr = 0.00001)

        
                         
                         
    #R-Net

    def r_net():
        model = Sequential() #there are 2 types of models but this is the most common
        model.add(Conv2D(28, (3, 3), strides=1, name='conv1', input_shape =(24, 24, 3)))
        model.add(PReLU(name='prelu1'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

        model.add(Conv2D(48, (3, 3), strides=1, name='conv2'))
        model.add(PReLU(name='prelu2'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

        model.add(Conv2D(64, (2, 2), strides=1, name='conv3'))
        model.add(PReLU(name='prelu3'))

        model.add(Dense(128))
        model.add(PReLU(name='prelu4'))

        model.add(Dense(2))
        model.add(Activation('softmax'))

        model.add(Dense(4))#THere is something being used input_layer_name='prelu4)

    #O-Net

    def o_net():
        model = Sequential() #there are 2 types of models but this is the most common
        model.add(Conv2D(32, (3, 3), strides=1, name='conv1'), input_shape =(48, 48, 3))
        model.add(PReLU(name='prelu1'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

        model.add(Conv2D(64, (3, 3), strides=1, name='conv2'))
        model.add(PReLU(name='prelu2'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

        model.add(Conv2D(64, (3, 3), strides=1, name='conv3'))
        model.add(PReLU(name='prelu3'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

        model.add(Conv2D(128, (2, 2), strides=1, name='conv4'))
        model.add(PReLU(name='prelu4'))

        model.add(Dense(256))
        model.add(PReLU(name='prelu5'))

        model.add(Dense(2))
        model.add(Activation('softmax'))

        model.add(Dense(4))#THere is something being used input_layer_name='prelu5

        model.add(Dense(10))#THere is something being used input_layer_name='prelu5

In [ ]:
def read_single_tfrecord(tfrecord_file, batch_size, net):
    # generate a input queue
    # each epoch shuffle
    filename_queue = tf.string_input_producer([tfrecord_file],shuffle=True)
    # read tfrecord
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    image_features = tf.parse_single_example(
        serialized_example,
        features={
            'image/encoded': tf.FixedLenFeature([], tf.string),#one image  one record
            'image/label': tf.FixedLenFeature([], tf.int64),
            'image/roi': tf.FixedLenFeature([4], tf.float32),
            'image/landmark': tf.FixedLenFeature([10],tf.float32)
        }
    )
    if net == 'PNet':
        image_size = 12
    elif net == 'RNet':
        image_size = 24
    else:
        image_size = 48
    image = tf.decode_raw(image_features['image/encoded'], tf.uint8)
    image = tf.reshape(image, [image_size, image_size, 3])
    image = (tf.cast(image, tf.float32)-127.5) / 128
    
    # image = tf.image.per_image_standardization(image)
    label = tf.cast(image_features['image/label'], tf.float32)
    roi = tf.cast(image_features['image/roi'],tf.float32)
    landmark = tf.cast(image_features['image/landmark'],tf.float32)
    image, label,roi,landmark = tf.train.batch(
        [image, label,roi,landmark],
        batch_size=batch_size,
        num_threads=2,
        capacity=1 * batch_size
    )
    label = tf.reshape(label, [batch_size])
    roi = tf.reshape(roi,[batch_size,4])
    landmark = tf.reshape(landmark,[batch_size,10])
    return image, label, roi,landmark